**Calculate TD shot gather from FD modelling results**<br />
Daniel Köhn<br />
Kiel, 01.09.2017

**Import necessary packages**

In [103]:
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
from matplotlib.pyplot import gca
import matplotlib as mpl
from pylab import rcParams
from matplotlib import rc
from scipy import interpolate
from scipy import signal
#%matplotlib notebook

**Define parameters for data visualization**

In [104]:
clip = 5e-3   # data clip
nsrc = 50      # number of shots
ntr = 25     # number of receivers
shotno = 1    # extract FDFD data for shot shotno

# number of frequencies nf and frequency interval df
FC_low = 50e7
FC_high = 400e7

nf = 1
df = (FC_high - FC_low) / (nf)
nfnsrc = nf * nsrc

# TD parameters
TmaxTD = 150e-9       # maximum time TD
TmaxFD = 0.25 / df    # maximum time FD
dt = TmaxFD / nf      # time sampling FD

# maximum time sample of FD2TD corresponding to TmaxTD
nmaxFD = np.int(TmaxTD / dt)

# define time samples in time and frequency domain
tFD = np.arange(0.0, TmaxTD, dt)
offset = np.arange(0, ntr, 1)

/tmp/ipykernel_3444/1558264301.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  nmaxFD = np.int(TmaxTD / dt)


**Font properties**

In [105]:
FSize = 25
font = {'color':  'black',
        'weight': 'bold',
        'size': FSize}
mpl.rc('xtick', labelsize=FSize) 
mpl.rc('ytick', labelsize=FSize) 
rcParams['figure.figsize'] = 10, 10

__Time Damping__

In [106]:
S = 0.0

**Read 2 cross TE-mode FD data**

In [107]:
name_FDFD = "../seis/MRTO_true/MRTO_p_stage_7.bin"
f = open (name_FDFD)
data_type = np.dtype ('float32').newbyteorder ('<')
data_FDFD = np.fromfile(f, dtype=data_type)
data_FDFD = data_FDFD.reshape(nf*nsrc,(2*ntr)-2)
data_FDFD = np.transpose(data_FDFD)
data_FDFD = np.flipud(data_FDFD)

**Extract FD data for shot shotno**

In [108]:
FD_real = data_FDFD[0:2*ntr-1:2,shotno-1:nfnsrc:nsrc]
FD_imag = data_FDFD[1:2*ntr:2,shotno-1:nfnsrc:nsrc]
data_FDFD = None

# taper FD data
window = signal.tukey(nf,1)

for i in range (ntr):
    FD_real[i-1,:] *= window
    FD_imag[i-1,:] *= window

In [109]:
# assemble complex data
FDFD = FD_real + 1j*FD_imag
FD_real = None
FD_imag = None
FDFD = np.transpose(FDFD)

# tmp1 = np.concatenate((FDFD, np.zeros((nf, ntr)), np.zeros((nf, ntr)), np.flipud(FDFD)))
tmp1 = np.concatenate((np.zeros((1, ntr-1)),FDFD,np.flipud(FDFD)))
FDFD = None
FDFD = tmp1

In [110]:
#plt.imshow(np.real(FDFD))
#plt.show()

**IFFT of FD data**

In [111]:
# transformation from FD to TD
FD2TD = np.fft.ifft(FDFD,axis=0)
FD2TD = np.real(FD2TD)
tmp1 = None

**Extract FD2TD up to TmaxTD and resample FD2TD from dtFD to dt**

In [112]:
# extract FD2TD data up to TmaxTD
tmp1 = FD2TD[1:nmaxFD,:]
FD2DTD = None

# clean memory
tmp = None

In [113]:
#plt.imshow(FD2TD, vmin=-clip, vmax=clip)
#plt.show()

In [114]:
#print(np.shape(tFD), np.shape(offset), np.shape(tmp1))

__Damp Time Domain Data__

In [115]:
#tmp2 = np.exp(-S * tTD)
#tdamp = np.tile(tmp2,(ntr,1))
#tdamp = np.flipud(tdamp.T)
#traces = traces * tdamp

#tmp2 = None
#tdamp = None

**Normalize data**

In [116]:
#max_TD = np.max(traces)
max_FD = np.max(tmp1)
tmp1 = tmp1 / max_FD
FD2TD = np.flipud(tmp1)

tmp1 = None

__Shuffle 2-cross TE model data__

In [117]:
tmp1 = FD2TD[:,0:29]
tmp2 = FD2TD[:,30:57]
tmp3 = FD2TD[:,58:87]
tmp4 = FD2TD[:,88:116]

FD2TD = None

tmp = np.concatenate((tmp1.T, tmp4.T, np.flipud(tmp2.T), np.flipud(tmp3.T)))
FD2TD = tmp.T

tmp = None
tmp1 = None
tmp2 = None
tmp3 = None
tmp4 = None

# plt.imshow(FD2TD)
# plt.show()

**Plot shot gather**

In [118]:
def do_plot(n, model, cm, an, title, clip):
    
    extent = [np.min(offset), np.max(offset), 0.0, dt * nmaxFD * 1e9]

    plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
    plt.rc('text', usetex=True)

    im = plt.imshow(model, cmap=cm, interpolation='nearest', extent=extent, vmin=-clip, vmax=clip, aspect=0.5)

    # a = gca()
    # a.set_xticklabels(a.get_xticks(), font)
    # a.set_yticklabels(a.get_yticks(), font)

    plt.title(title, fontdict=font)
    if n==1:
        plt.ylabel('Time [ns]', fontdict=font)
    plt.xlabel('Trace no.', fontdict=font)
    plt.gca().invert_yaxis()
    plt.savefig("test.png", format='png')

In [119]:
do_plot(1, FD2TD, 'gray', '(b)', r"2-cross TE-mode model (GERMAINE)", clip)
plt.tight_layout()
output_file = "test.png"
plt.savefig(output_file, bbox_inches='tight', format='png')
plt.show()

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.20 (TeX Live 2019/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(/home/yubel/.cache/matplotlib/tex.cache/a0dd76c033a72e676ddb61a79c5e4248.tex
LaTeX2e <2020-02-02> patch level 2
L3 programming layer <2020-02-14>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2019/12/20 v1.4l Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))

! LaTeX Error: File `type1cm.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.5 \usepackage
               {type1ec}^^M
No pages of output.
Transcript written on a0dd76c033a72e676ddb61a79c5e4248.log.




RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.20 (TeX Live 2019/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(/home/yubel/.cache/matplotlib/tex.cache/a0dd76c033a72e676ddb61a79c5e4248.tex
LaTeX2e <2020-02-02> patch level 2
L3 programming layer <2020-02-14>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2019/12/20 v1.4l Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))

! LaTeX Error: File `type1cm.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.5 \usepackage
               {type1ec}^^M
No pages of output.
Transcript written on a0dd76c033a72e676ddb61a79c5e4248.log.




<Figure size 720x720 with 1 Axes>